In [2]:
!pip install geopy
from geopy.geocoders import Nominatim
import geopandas as gpd
import pandas as pd
import plotly.express as px
from shapely.geometry import Point


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python3 -m pip install --upgrade pip


ModuleNotFoundError: No module named 'geopandas'

In [3]:
import pandas as pd
import io

nelex_df = pd.read_csv('nelex_NGA.csv')
print(nelex_df)

FileNotFoundError: [Errno 2] No such file or directory: 'nelex_NGA.csv'

In [48]:
!pip install geopy geopandas plotly
from geopy.geocoders import Nominatim
import geopandas as gpd
import pandas as pd
import plotly.express as px
from shapely.geometry import Point


In [49]:
# Initialize geolocator
geolocator = Nominatim(user_agent="nigeria_map")

# Function to geocode addresses
def geocode_address(address):
    try:
        location = geolocator.geocode(address)
        return location.latitude, location.longitude
    except:
        return None, None

# Geocode NELEX addresses
nelex_df[['latitude', 'longitude']] = nelex_df['nelex_address'].apply(
    lambda x: pd.Series(geocode_address(x))
)

# Drop any rows where geocoding failed
nelex_df.dropna(subset=['latitude', 'longitude'], inplace=True)

# Convert to GeoDataFrames
nelex_df['geometry'] = nelex_df.apply(lambda row: Point(row['longitude'], row['latitude']), axis=1)
nelex_gdf = gpd.GeoDataFrame(nelex_df, geometry='geometry')

In [45]:
# Load Nigeria shapefile
nigeria_gdf = gpd.read_file("gadm41_NGA_0.shp")

In [55]:
# Convert geometries to latitude and longitude for Plotly
nelex_df['lon'] = nelex_df['geometry'].apply(lambda point: point.x)
nelex_df['lat'] = nelex_df['geometry'].apply(lambda point: point.y)

# Plot using Plotly
# Removed geojson and locations arguments, and use mapbox_style='open-street-map'
# as originally intended in update_layout, also using lat and lon for Nigeria data
fig = px.scatter_mapbox(
    nigeria_gdf,  # Use nigeria_gdf for the base map
    lat=nigeria_gdf.geometry.centroid.y,  # Latitude for Nigeria (using centroid)
    lon=nigeria_gdf.geometry.centroid.x,  # Longitude for Nigeria (using centroid)
    #color_discrete_sequence=["#E3DCDC"], # Removing color for base map
    center={"lat": 9.0820, "lon": 8.6753},  # Center on Nigeria
    zoom=5,
    mapbox_style="open-street-map",  # Directly set the map style here
    hover_data=['COUNTRY'] # Display the name of Nigeria on hover, changed from 'NAME_0' to 'COUNTRY'
)

# Add NELEX centers to the map as a separate trace
fig.add_trace(
    px.scatter_mapbox(
        nelex_df,
        lat="lat",
        lon="lon",
        color_discrete_sequence=["green"], # Distinct color for NELEX markers
        hover_data=['nelex_office'] # Display NELEX center name on hover
    ).data[0]
)

# Update layout for better visualization
# Remove redundant mapbox style setting
fig.update_layout(
    title="Locations of NELEX Centers in Nigeria",
    mapbox=dict(
        #style="open-street-map", # Already set in the px.scatter_mapbox call
        zoom=5
    ),
    legend=dict(
        title="Legend",
        orientation="h"
    )
)

fig.show()

<ipython-input-55-fbde84e0ff62>:10: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.


<ipython-input-55-fbde84e0ff62>:11: UserWarning:

Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.


